In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [2]:
data_row = pd.read_csv("D:\\Big Data Science NTI\\Project\\Dataset\\instacart_2017_05_01\\order_products__prior.csv")
data = data_row.head(7000000)

In [3]:
data.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [4]:
data.describe()

,order_id,product_id,add_to_cart_order,reordered
count,7.000000e+06,7.000000e+06,7.000000e+06,7.000000e+06
mean,3.694217e+05,2.557978e+04,8.345091e+00,5.891573e-01
std,2.133013e+05,1.409828e+04,7.124295e+00,4.919868e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,1.845428e+05,1.352400e+04,3.000000e+00,0.000000e+00
50%,3.697780e+05,2.525600e+04,6.000000e+00,1.000000e+00
75%,5.540410e+05,3.794000e+04,1.100000e+01,1.000000e+00
max,7.390580e+05,4.968800e+04,1.270000e+02,1.000000e+00


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000000 entries, 0 to 6999999
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 213.6 MB


In [6]:
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [7]:
print('orders -- dimensions: {0};   size: {1}'.format(data.shape, size(data)))

orders -- dimensions: (7000000, 4);   size: 224.00 MB


In [8]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
orders = data.set_index('order_id')['product_id'].rename('item_id')

In [9]:
display(orders.head(20))
type(orders)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
3    24838
3    17704
3    21903
3    17668
3    46667
3    17461
3    32665
4    46842
4    26434
4    39758
Name: item_id, dtype: int64

pandas.core.series.Series

In [10]:
print('dimensions: {0};   size: {1};   unique_orders: {2};   unique_items: {3}'
      .format(orders.shape, size(orders), len(orders.index.unique()), len(orders.value_counts())))

dimensions: (7000000,);   size: 112.00 MB;   unique_orders: 694184;   unique_items: 47470


In [11]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")


In [12]:
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))

In [13]:
# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair

In [14]:
# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))

In [15]:
# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]      

In [16]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats = freq(order_item).to_frame("freq")
    item_stats['support'] = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items = item_stats[item_stats['support'] >= min_support].index
    order_item = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size = freq(order_item.index)
    qualifying_orders = order_size[order_size >= 2].index
    order_item = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats = freq(order_item).to_frame("freq")
    item_stats['support'] = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [17]:
%%time
rules = association_rules(orders, 0.04)  

Starting order_item:                7000000
Items with support >= 0.04:            4123
Remaining order_item:               5494313
Remaining orders with 2+ items:      627718
Remaining order_item:               5443208
Item pairs:                         6422989
Item pairs with support >= 0.04:       6530

Wall time: 10min 29s


In [18]:
# Replace item ID with item name and display association rules
item_name   = pd.read_csv('D:\\Big Data Science NTI\\Project\\Dataset\\instacart_2017_05_01\\products.csv')
item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Almond Milk Strawberry Yogurt,Almond Milk Blueberry Yogurt,365,0.058147,1253,0.199612,1036,0.165042,0.291301,0.352317,1.765008
1,Almond Milk Peach Yogurt,Almond Milk Blueberry Yogurt,289,0.046040,1049,0.167113,1036,0.165042,0.275500,0.278958,1.669272
2,Almond Milk Strawberry Yogurt,Almond Milk Peach Yogurt,308,0.049067,1253,0.199612,1049,0.167113,0.245810,0.293613,1.470919
3,Almond Milk Peach Yogurt,Almond Milk Strawberry Yogurt,256,0.040783,1049,0.167113,1253,0.199612,0.244042,0.204310,1.222582
4,Yotoddler Organic Pear Spinach Mango Yogurt,Organic Whole Milk Strawberry Beet Berry Yogur...,360,0.057351,1317,0.209808,1412,0.224942,0.273349,0.254958,1.215197
5,Strawberry on the Bottom Nonfat Greek Yogurt,Blueberry on the Bottom Nonfat Greek Yogurt,357,0.056873,1649,0.262698,1354,0.215702,0.216495,0.263663,1.003676
9,Organic Whole Milk Strawberry Beet Berry Yogur...,Yotoddler Organic Pear Spinach Mango Yogurt,295,0.046996,1412,0.224942,1317,0.209808,0.208924,0.223994,0.995786
10,Organic Greek Nonfat Yogurt With Mixed Berries,Organic Nonfat Greek Yogurt With Peaches,265,0.042216,1459,0.232429,1286,0.204869,0.181631,0.206065,0.886572
6,Peach on the Bottom Nonfat Greek Yogurt,Blueberry on the Bottom Nonfat Greek Yogurt,311,0.049545,1685,0.268433,1354,0.215702,0.184570,0.229690,0.855670
11,Mixed Berries Whole Milk Icelandic Style Skyr ...,Strawberry Rhubarb Yogurt,298,0.047474,1612,0.256803,1408,0.224305,0.184864,0.211648,0.824163


In [19]:
rules_final.to_csv("final_rules.csv", sep=',', encoding='utf-8')

In [20]:
len(rules_final)

6530